# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier():
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.params = {'min_samples_split': min_samples_split,
                       'max_depth': max_depth,
                       'criterion': criterion,
                       'sufficient_share': sufficient_share,
                       'max_features': max_features}

        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s_1, r_s_1 = l_s.reshape(l_s.shape[0]), r_s.reshape(r_s.shape[0])
        l_part, r_part = (l_s_1 / (l_s_1 + r_s_1)), (r_s_1 / (l_s_1 + r_s_1))
        return ((np.ones(l_s.shape[1]) - np.sum(np.square(l_c / l_s), axis=-1))
                * l_part +
                (np.ones(r_s.shape[1]) - np.sum(np.square(r_c / r_s), axis=-1))
                * r_part)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s_1, r_s_1 = l_s.reshape(l_s.shape[0]), r_s.reshape(r_s.shape[0])
        l_part, r_part = (l_s_1 / (l_s_1 + r_s_1)), (r_s_1 / (l_s_1 + r_s_1))
        p_l, p_r = l_c / l_s, r_c / r_s
        # несмотря на where кидает предупреждение о log(0)
        return ((-np.sum(np.where(p_l == 0, p_l, p_l * np.log(p_l)))) * l_part
                +
                (-np.sum(np.where(p_r == 0, p_r, p_r * np.log(p_r)))) * r_part)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s_1, r_s_1 = l_s.reshape(l_s.shape[0]), r_s.reshape(r_s.shape[0])
        l_part, r_part = (l_s_1 / (l_s_1 + r_s_1)), (r_s_1 / (l_s_1 + r_s_1))
        p_l = np.append(l_c / l_s, np.zeros(l_s.shape), axis=-1)
        p_r = np.append(r_c / r_s, np.zeros(r_s.shape), axis=-1)
        return ((np.ones(l_s.shape[1]) - np.max(p_l, axis=-1)) * l_part +
                (np.ones(r_s.shape[1]) - np.max(p_r, axis=-1)) * r_part)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.array(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:max(1, int(np.ceil(np.sqrt(n_feature))))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.array(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:max(1, int(np.ceil(np.log2(n_feature))))]

    def __get_feature_ids_N(self, n_feature):
        return np.array(range(n_feature))

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # - сортирует объекты по фиче x, считает минимальное число объектов
        #   в одной из вершин разбиения
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        # - находит отрезки из одинаковых объектов, сохраняет их границы;
        #   если в центре (без хвостов) все объекты одинаковые,
        #   то разделить нельзя, возвращаем impurity = inf
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        # - считает для каждого разбиения по одному из объектов из r_border_ids
        #   сколько объектов какого класса попадут левее данного объекта
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # - считает для каждого разбиения по одному из border_ids сколько
        #   объектов какого класса попадут в левую и правую вершины, а также
        #   общее количество объектов в разбиениях
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # - вычисляет impurity каждого разбиения,
        #   находит оптимальное с min impurity
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #####
        # проверка на совпадающие иксы и невозможность разделить
        # без неё почему-то работает в 50 раз быстрее, но точность
        # на SpeedDating становится меньшь 50%...
        ####
        gs = gs[sorted_x[l_sizes[:, 0] - 1] != sorted_x[l_sizes[:, 0]]]
        if gs.shape[0] == 0:
            return np.inf, None
        #####
        # работает быстрее, потому что на первом же разделении натыкается на
        # разбиение, где в одном листе пусто, и корень сразу объявляется листом...
        #####
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # - возвращает impurity и границу разбиения
        #   для полученного оптимального разбиения
        left_el_id = l_sizes[idx][0]
        border = (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __make_leaf(self, node_id, features, shares):
        feature_id = features[0][np.argmax(shares)]
        node = (self.LEAF_TYPE, feature_id, 0.0)
        self.tree[node_id] = node
        return

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE     _, feature_id, threshold = node
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

        features = np.unique(y, return_counts=True)
        shares = features[1] / np.sum(features[1])
        if ((self.max_depth is not None and depth >= self.max_depth) or
                (np.max(shares) >= self.sufficient_share) or
                (y.shape[0] <= self.min_samples_split)):
            self.__make_leaf(node_id, features, shares)
            return

        feature_ids = self.get_feature_ids(x.shape[1])

        splits = [(*self.__find_threshold(x[:, f_id], y), f_id)
                  for f_id in feature_ids if f_id != pred_f]
        # нужно сравнивать кортежи целиком
        gs, threshold, feature_id = min(splits)

        if (threshold is None):
            self.__make_leaf(node_id, features, shares)
            return

        pred_gs = self.G_function(
            np.array([np.unique(y, return_counts=True)[1]]),
            np.array([[y.shape[0]]]),
            np.array([[]]), np.array([[0]]))

        self.feature_importances_[feature_id] += (pred_gs - gs)

        node = (self.NON_LEAF_TYPE, feature_id, threshold)
        self.tree[node_id] = node

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)

        if (y_l.shape[0] == 0 or y_r.shape[0] == 0):
            self.__make_leaf(node_id, features, shares)
            return

        self.__fit_node(x_l, y_l, node_id * 2 + 1, depth + 1, feature_id)
        self.__fit_node(x_r, y_r, node_id * 2 + 2, depth + 1, feature_id)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def score(self, X_test, y_test):
        return f1_score(y_pred=self.predict(X_test),
                        y_true=y_test, average='macro')

    def get_params(self, deep=True):
        return self.params

    def set_params(self, **params):
        self.__init__(**params)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 1e+03 µs


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 24 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [9]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', 
                 encoding='cp1251')


In [10]:
df = df.iloc[:, :97]

In [11]:
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)

In [12]:
df = df.drop(['condtn'], axis=1)

In [13]:
df = df.drop(['round'], axis=1)

In [14]:
df = df.drop(['position', 'positin1'], axis=1)

In [15]:
df = df.drop(['order'], axis=1)

In [16]:
df = df.drop(['partner'], axis=1)

In [17]:
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)

In [18]:
df = df.dropna(subset=['age'])

In [19]:
df = df.drop(['field'], axis=1)

In [20]:
df = df.drop(['undergra'], axis=1)

In [21]:
# Что будем делать?
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

In [22]:
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)

In [23]:
# Что будем делать?
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

In [24]:
# Что делать?

df = df.dropna(subset=['imprelig', 'imprace'])

In [25]:
df = df.drop(['from', 'zipcode'], axis=1)

In [26]:
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

In [27]:
# df = df.drop(['income'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

In [28]:
df = df.dropna(subset=['date'])

In [29]:
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)

In [30]:
df = df.drop(['career'], axis=1)

In [31]:
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)

In [32]:
df = df.drop(['expnum'], axis=1)

In [33]:
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']

In [34]:
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

In [35]:
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [36]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)

In [37]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))

In [38]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)

In [39]:
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

### Attr2

In [40]:
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

In [41]:
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

In [42]:
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [43]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)

In [44]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))

In [45]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)

In [46]:
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

In [47]:
df = df.drop(['temp_totalsum'], axis=1)

In [48]:
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)

In [49]:
df = df.drop(['wave'], axis=1)

In [50]:
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

In [51]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

In [52]:
df_female = df_female.drop(['pid_f'], axis=1)

In [53]:
df_pair = df_male.join(df_female.set_index('iid_f'), 
                       on='pid', 
                       how='inner')

In [54]:
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [55]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
df_pair.head()

,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,...,sinc2_1_f,intel2_1_f,fun2_1_f,amb2_1_f,shar2_1_f,attr3_1_f,sinc3_1_f,fun3_1_f,intel3_1_f,amb3_1_f
3489,0,0.43,0,27.0,16.0,1450.0,26100.0,2.0,2.0,3.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3498,0,0.46,0,28.0,8.0,1155.0,13258.0,2.0,8.0,9.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3507,0,0.05,0,32.0,7.0,1430.0,26908.0,2.0,1.0,6.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3516,0,0.44,0,25.0,10.0,1140.0,9168.0,2.0,7.0,1.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0
3534,0,0.23,1,27.0,8.0,1360.0,26062.0,6.0,1.0,1.0,...,1.0,1.0,3.0,10.0,10.0,7.0,7.0,8.0,8.0,10.0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)

## Проверка скорости работы на Speed Dating Data 

In [57]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
%time clf.fit(X_train, y_train)

Wall time: 12 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [70]:
# параметры из GridSearchCV
my_clf.set_params(**{'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 4})

In [71]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
%time my_clf.fit(X_train, y_train)

Wall time: 260 ms


## Проверка качества работы на Speed Dating Data

In [72]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.57328385899814471

In [73]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.60949988581868009

## Задание 3

In [62]:
best_clf = sorted(zip(df_pair.keys()[1:], clf.feature_importances_), key = lambda x: -x[1])
print("Best clf features:")
for feature in best_clf[:10]:
    print(feature[0], ':', feature[1])
print()

best_my_clf = sorted(zip(df_pair.keys()[1:], my_clf.feature_importances_), key = lambda x: -x[1])
print("Best my_clf features:")
for feature in best_my_clf[:10]:
    print(feature[0], ':', feature[1])
print()

Best clf features:
int_corr : 0.0687644546359
field_cd : 0.0544632098921
amb3_1_f : 0.0488464590217
fun2_1 : 0.0396190460126
imprelig : 0.038623118189
shar2_1 : 0.0380803436396
shar1_1 : 0.0350521039305
amb1_1_f : 0.0338577648424
tuition_f : 0.0304488457531
amb1_1 : 0.0303318856964

Best my_clf features:
int_corr : 0.109703966453
field_cd_f : 0.0615813368467
fun3_1 : 0.0548815154768
shar1_1 : 0.0478295449812
imprace_f : 0.0473976724572
date_f : 0.0470096213377
amb1_1 : 0.0430649212354
field_cd : 0.0419095209096
amb2_1_f : 0.0368435348656
career_c_f : 0.0342593702673



## Задание 4

In [63]:
params = {'min_samples_split': list(range(2, 5)), 
          'max_depth': [None] + list(range(3, 6)),
          'criterion': ['gini', 'entropy'], 
          'max_features': [None, 'sqrt', 'log2']}
optimiser = GridSearchCV(DecisionTreeClassifier(), params)
optimiser.fit(X, y)
print(optimiser.best_params_)
print(optimiser.best_score_)

{'criterion': 'entropy', 'max_depth': 3, 'max_features': 'log2', 'min_samples_split': 2}
0.857782754759


In [64]:
params = {'min_samples_split': list(range(2, 5)), 
          'max_depth': [None] + list(range(3, 6)),
          'criterion': ['gini', 'entropy', 'misclass'], 
          'max_features': [None, 'sqrt', 'log2']}
# optimiser = GridSearchCV(MyDecisionTreeClassifier(), params,refit=False)
optimiser = GridSearchCV(MyDecisionTreeClassifier(), params,refit=False)
optimiser.fit(X, y)
print(optimiser.best_params_)
print(optimiser.best_score_)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in multiply
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in multiply


{'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 4}
0.521316389781


In [65]:
# если закомментировать кусок кода, выкидывающий часть gs, которые невозможно получить, 
# то максимальная точность получается меньше 50%, что совсем плохо для бинарного классификатора,
# но fit выполняется около 16ms против 13ms у sklearn